<!-- # Compare Runs, Choose a Model, Deploy the model to a REST API -->

    -Run Hyperparameter Sweep on Training Script
    -Compare the Results of Runs on MLFlow UI
    -Choose the best Model and Register it as a Model
    -Deploy the Model on REST API
    -Build a Container Image suitable for Deployment in a Cloud Platform

In [62]:
import numpy as np
import pandas as pd
import keras
from sklearn.model_selection import train_test_split
from hyperopt import STATUS_OK,Trials,fmin,hp,tpe
from sklearn.metrics import mean_squared_error
import mlflow
from mlflow.models import infer_signature

data_df = pd.read_csv("https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv", sep=";")
data_df.head()


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [63]:
x_train = data_df.drop(columns='quality',axis=1).values
y_train = data_df['quality'].values.ravel()

y_test = data_df.drop(columns='quality',axis=1).values
x_test = data_df['quality'].values.ravel()

In [64]:
x_train,x_valid,y_train,y_valid = train_test_split(x_train,y_train,test_size=0.2,random_state=42)


In [65]:
#ANN Model

import mlflow.tensorflow
import mlflow.tensorflow


def train_model(params, epochs,  train_x, train_y, valid_x, valid_y, test_x, test_y):
    #Model Architecture
    mean = np.mean(train_x,axis=0)
    var = np.var(train_x,axis=0)

    model = keras.Sequential(
        [
            keras.Input([x_train.shape[1]]),
            keras.layers.Normalization(mean=mean,variance=var),
            keras.layers.Dense(64, activation="relu"),
            keras.layers.Dense(1)
        ]
    )

    #Compile Model
    model.compile(optimizer=keras.optimizers.SGD(
        learning_rate=params["lr"], momentum=params["momentum"]

    ),
    loss="mean_squared_error",
    metrics=[keras.metrics.RootMeanSquaredError()]
    )

    #Train ANN Model with LR and Momentum params with MLFlow Tracking 
    mlflow.set_experiment("Wine Quality: ANN Model Selection --ALL MODELS")
    with mlflow.start_run(nested=True):
        model.fit(x_train,y_train,validation_data=(x_valid,y_valid), epochs = epochs, batch_size=65)
        
        #Evaluate Model
        eval_result = model.evaluate(valid_x,valid_y,batch_size=65)
        eval_rmse = eval_result[1]

        
        #Log Parameters and Results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse", eval_rmse)

        #Log the Model
        from mlflow.models import infer_signature
        signature = infer_signature(train_x, model.predict(train_x[:5]))
        mlflow.keras.log_model(model, artifact_path="model", signature=signature)

        return {"loss" : eval_rmse, "status" : STATUS_OK, "model" : model }




In [66]:
def objective(params):
    #MLFlow will track the parameters and results for each run
     result = train_model(
          params,
          epochs=10,
          train_x=x_train,
          train_y=y_train,
          valid_x=x_valid,
          valid_y=y_valid,
          test_x=x_test,
          test_y=y_test
     )
     return result

In [70]:
space = {
    "lr":hp.loguniform("lr",np.log(1e-5),np.log(1e-2)),
    "momentum":hp.uniform("momentum",0.7,1)
    

}

In [71]:
import mlflow.keras
import mlflow.keras
import mlflow.tensorflow
import mlflow.tensorflow


mlflow.set_experiment("Wine Quality: ANN Model Selection --Best Models")
with mlflow.start_run():
    #HyperParameter Search using Hyperopt
    trials = Trials()
    best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=50,
        trials=trials
    )
    

    #Details of Best Run
    best_run = sorted(trials.results,key=lambda x: x["loss"])[0]

    #Log Best Parameters, Loss, and Model
    mlflow.log_params(best)
    mlflow.log_metric("eval mse", best_run["loss"])
    from mlflow.models import infer_signature
    signature = infer_signature(x_train, best_run["model"].predict(x_train[:5]))
    mlflow.keras.log_model(best_run["model"], artifact_path="model", signature=signature)

    #Print best params
    print(f"Best parameters: {best}")
    print(f"Best eval rsme: {best_run["loss"]}")
    print(f"Worst eval rsme: {sorted(trials.results,key=lambda x: x["loss"])[-1]}")


  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

Epoch 1/10                                            

 1/61 ━━━━━━━━━━━━━━━━━━━━ 11s 188ms/step - loss: 34.4457 - root_mean_squared_error: 5.8690
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 17.2920 - root_mean_squared_error: 4.0449 - val_loss: 2.1934 - val_root_mean_squared_error: 1.4810

Epoch 2/10                                            

 1/61 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 2.1287 - root_mean_squared_error: 1.4590
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 765us/step - loss: 1.8128 - root_mean_squared_error: 1.3459 - val_loss: 1.6481 - val_root_mean_squared_error: 1.2838

Epoch 3/10                                            

 1/61 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 1.4934 - root_mean_squared_error: 1.2220
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 733us/step - loss: 1.3386 - root_mean_squared_error: 1.1568 - val_loss: 1.3788 - val_root_mean_squared_error: 1.1742

Epoch 4/10                                            

 1/61 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 1.2130 - root_mean

2025/07/13 14:32:07 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Epoch 1/10                                                                      

 1/61 ━━━━━━━━━━━━━━━━━━━━ 11s 200ms/step - loss: 33.5306 - root_mean_squared_error: 5.7906
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 31.4000 - root_mean_squared_error: 5.5968 - val_loss: 16.8165 - val_root_mean_squared_error: 4.1008

Epoch 2/10                                                                      

 1/61 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 17.4442 - root_mean_squared_error: 4.1766
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 838us/step - loss: 13.9096 - root_mean_squared_error: 3.7222 - val_loss: 7.2791 - val_root_mean_squared_error: 2.6980

Epoch 3/10                                                                      

 1/61 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 6.5945 - root_mean_squared_error: 2.5680
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 837us/step - loss: 6.0368 - root_mean_squared_error: 2.4548 - val_loss: 3.8421 - val_root_mean_squared_error: 1.9601

Epoch 4/10                            

2025/07/13 14:32:14 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



...

Epoch 1/10                                                                       

 1/61 ━━━━━━━━━━━━━━━━━━━━ 8s 141ms/step - loss: 37.2933 - root_mean_squared_error: 6.1068
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 10.5123 - root_mean_squared_error: 3.1071 - val_loss: 0.7380 - val_root_mean_squared_error: 0.8591

Epoch 2/10                                                                       

 1/61 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.5322 - root_mean_squared_error: 0.7295
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 743us/step - loss: 0.7490 - root_mean_squared_error: 0.8645 - val_loss: 0.5519 - val_root_mean_squared_error: 0.7429

Epoch 3/10                                                                       

 1/61 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.3611 - root_mean_squared_error: 0.6009
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 713us/step - loss: 0.5437 - root_mean_squared_error: 0.7371 - val_loss: 0.5177 - val_root_mean_squared_error: 0.7195

Epoch 4/10                              

2025/07/13 14:41:17 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



100%|██████████| 50/50 [09:15<00:00, 11.10s/trial, best loss: 0.6956303119659424]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step


2025/07/13 14:41:22 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Best parameters: {'lr': 0.007442004941898027, 'momentum': 0.9485609710164002}
Best eval rsme: 0.6956303119659424
Worst eval rsme: {'loss': nan, 'status': 'ok', 'model': <Sequential name=sequential_201, built=True>}


In [72]:
print(best_run["model"])


<Sequential name=sequential_194, built=True>
